In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import pandas as pd
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M24072'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[[20241203]]
base_folder='/mnt/rds01/ibn-vision/DATA/SUBJECTS/' 
no_probe = 1
# slice off the reocrding that had failing errors happened during the recording
#acquisition_to_slice = []
#slice_time_ranges = []
no_dates = len(save_date_list)


date_count = 0
for save_date_tmp in save_date_list:
    save_date = str(save_date_tmp[0])
    recording_folder = base_folder + mouse + '/ephys/' + save_date + '/'
    error_info_path = os.path.join(recording_folder, 'error_info.csv')
    if os.path.exists(error_info_path):
        error_info = pd.read_csv(error_info_path)
        print(error_info)
    else:
        print(f'No error_info.csv found in {recording_folder}')
    num_rows = len(error_info)
    if num_rows > 0:
        print('error exists in recordings. Will slice recordings after error happened')
        unique_acquisition_indices = error_info.drop_duplicates(subset='acquisition', keep='first').index.tolist()
        print(f'Unique acquisition indices: {unique_acquisition_indices}')
        acquisition_to_slice = error_info['acquisition'].tolist()
        segment_to_slice = error_info['session'].apply(lambda x: int(x.split('_')[2])).tolist()
        error_sample = error_info['error_sample'].tolist()
        acquisition_to_slice = [acquisition_to_slice[i] for i in unique_acquisition_indices]
        segment_to_slice = [segment_to_slice[i] for i in unique_acquisition_indices]
        error_sample = [error_sample[i] for i in unique_acquisition_indices]
    acquisition_folders = []

    acquisition_base_path = base_folder + mouse + '/ephys/' + save_date + '/*' + save_date
    print(acquisition_base_path)
    acquisition_folders = glob.glob(acquisition_base_path + '_*')
    print('acquisition_folders: ', acquisition_folders)

    for probe in range(int(no_probe)):
        for acquisition_folder in acquisition_folders:
                # Rename all tcat files to t0 if they exist
            tcat_pattern = os.path.join(acquisition_folder,'**','*tcat.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            # Step 1: Iterate over the list of files with tcat in the name
            for old_name in files_to_rename:
                # Step 2: Construct the new filename (REMEMBER to switch the name back to tcat)
                new_name = old_name.replace('tcat', 't0')
                
                # Step 3: Rename the file
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            
            recording_raw = si.read_spikeglx(acquisition_folder,stream_name='imec' + str(probe) + '.ap')
            no_segments = recording_raw.get_num_segments()
            segments = [i for i in range(no_segments)]
            num_segments = [recording_raw.get_num_frames(segment_index=i) for i in segments]
            end_sample = list(itertools.accumulate(num_segments))

            print('reading recording from:', acquisition_folder,'and probe',str(probe))
            if any(f'_{acq}' in acquisition_folder for acq in acquisition_to_slice):
                print(f'Acquisition {acquisition_folder} corresponds to one of the acquisition_to_slice: {acquisition_to_slice}')
                matching_acquisition = next((acq for acq in acquisition_to_slice if f'_{acq}' in acquisition_folder), None)
                slice_index = acquisition_to_slice.index(matching_acquisition)
                sample_index = error_sample[slice_index]
                segment_index = segment_to_slice[slice_index]
                print(f'Removing samples for acquisition {matching_acquisition} segment {segment_index} after {sample_index-300000} th sample')
                if matching_acquisition is not None:
                    print(f'Matching acquisition: {matching_acquisition}')
                
                selected_segment =  si.select_segment_recording(recording_raw,segment_indices=segment_index)
                recording_raw =selected_segment.frame_slice( start_frame=0, end_frame=sample_index-300000 )
            print('sliced recording:')
            print(recording_raw)
            print(recording_raw.get_num_segments)
            compression_folder = base_folder + mouse + '/ephys/' + save_date + '/probe' + str(probe) + '_compressed' + acquisition_folder[-2:]
            print('compressing to folder: '+compression_folder)
            raw_compressed = recording_raw.save(format="zarr", folder=compression_folder, compressor=compressor_wv, n_jobs=16, chunk_duration="1s")
            tcat_pattern = os.path.join(acquisition_folder, '**', '*t0.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            for old_name in files_to_rename:
                new_name = old_name.replace('t0', 'tcat')    
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            load_compressed = si.read_zarr(compression_folder+'.zarr')
            recording_raw.get_probe().to_dataframe()
            fig, ax = plt.subplots(figsize=(15, 10))
            si.plot_probe_map(recording_raw, ax=ax)
            ax.set_ylim(-100, 4000)
    date_count = date_count + 1

/home/saleem_lab/anaconda3/envs/si_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['imec1.ap', 'imec0.ap', 'nidq', 'imec1.lf', 'imec0.lf']
['imec1.ap', 'imec0.ap', 'nidq', 'imec1.lf', 'imec0.lf']
SpikeGLXRecordingExtractor: 384 channels - 30.0kHz - 5 segments - 371,182,730 samples 
                            12,372.79s (3.44 hours) - int16 dtype - 265.49 GiB
Segments:
Samples:   4,665,403 | 54,043,723 | 230,016,880 | 63,209,978 | 19,246,746
Durations: 155.51s (2.59 minutes) | 1,801.46s (30.02 minutes) | 7,667.25s (2.13 hours) | 2,107.00s (35.12 minutes) | 641.56s (10.69 minutes)
Memory:    3.34 GiB | 38.66 GiB | 164.52 GiB | 45.21 GiB | 13.77 GiB
SpikeGLXRecordingExtractor: 384 channels - 30.0kHz - 5 segments - 371,182,700 samples 
                            12,372.79s (3.44 hours) - int16 dtype - 265.49 GiB
Segments:
Samples:   4,665,374 | 54,043,751 | 230,016,879 | 63,209,979 | 19,246,717
Durations: 155.51s (2.59 minutes) | 1,801.46s (30.02 minutes) | 7,667.25s (2.13 hours) | 2,107.00s (35.12 minutes) | 641.56s (10.69 minutes)
Memory:    3.34 GiB | 38.66 GiB | 1

In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M24072'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[[20241206]]
base_folder='/mnt/rds01/ibn-vision/DATA/SUBJECTS/' 
no_probe = 1
# slice off the reocrding that had failing errors happened during the recording
acquisition_to_slice = []
slice_time_ranges = []
no_dates = len(save_date_list)


date_count = 0
for save_date_tmp in save_date_list:
    save_date = str(save_date_tmp[0])
    
    acquisition_folders = []
    acquisition_base_path = base_folder + mouse + '/ephys/' + save_date + '/*' + save_date
    print(acquisition_base_path)
    acquisition_folders = glob.glob(acquisition_base_path + '_*')
    print('acquisition_folders: ', acquisition_folders)

    for probe in range(int(no_probe)):
        for acquisition_folder in acquisition_folders:
                # Rename all tcat files to t0 if they exist
            tcat_pattern = os.path.join(acquisition_folder,'**','*tcat.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            # Step 1: Iterate over the list of files with tcat in the name
            for old_name in files_to_rename:
                # Step 2: Construct the new filename (REMEMBER to switch the name back to tcat)
                new_name = old_name.replace('tcat', 't0')
                
                # Step 3: Rename the file
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            
            recording_raw = si.read_spikeglx(acquisition_folder,stream_name='imec' + str(probe) + '.ap')
            print('reading recording from:', acquisition_folder,'and probe',str(probe))
            if any(f'_{acq}' in acquisition_folder for acq in acquisition_to_slice):
                print(f'Acquisition {acquisition_folder} corresponds to one of the acquisition_to_slice: {acquisition_to_slice}')
                matching_acquisition = next((acq for acq in acquisition_to_slice if f'_{acq}' in acquisition_folder), None)
                slice_index = acquisition_to_slice.index(matching_acquisition)
                time_window = slice_time_ranges[slice_index]
                print(f'Time window for acquisition {matching_acquisition}: {time_window}')
                if matching_acquisition is not None:
                    print(f'Matching acquisition: {matching_acquisition}')
                print('slicing recording to:', time_window)

                recording_raw =recording_raw.frame_slice( start_frame=time_window[0], end_frame=int(time_window[1] * 30000) )
            print(recording_raw)
            compression_folder = base_folder + mouse + '/ephys/' + save_date + '/probe' + str(probe) + '_compressed' + acquisition_folder[-2:]
            print('compressing to folder: '+compression_folder)
            raw_compressed = recording_raw.save(format="zarr", folder=compression_folder, compressor=compressor_wv, n_jobs=16, chunk_duration="1s")
            tcat_pattern = os.path.join(acquisition_folder, '**', '*t0.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            for old_name in files_to_rename:
                new_name = old_name.replace('t0', 'tcat')    
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            load_compressed = si.read_zarr(compression_folder+'.zarr')
            recording_raw.get_probe().to_dataframe()
            fig, ax = plt.subplots(figsize=(15, 10))
            si.plot_probe_map(recording_raw, ax=ax)
            ax.set_ylim(-100, 4000)
    date_count = date_count + 1

write_zarr_recording 
n_jobs=16 - samples_per_chunk=29,999 - chunk_memory=21.97 MiB - total_memory=351.55 MiB - chunk_duration=1.00s (999.97 ms)


write_zarr_recording: 100%|████████████████████████████████████████| 12376/12376 [54:20<00:00,  3.80it/s]


write_zarr_recording 
n_jobs=16 - samples_per_chunk=29,999 - chunk_memory=21.97 MiB - total_memory=351.55 MiB - chunk_duration=1.00s (999.97 ms)


write_zarr_recording: 100%|████████████████████████████████████████| 12376/12376 [59:15<00:00,  3.48it/s]


In [4]:
probe0_compressed_folder =dst_folder + '/probe0_raw_compressed.zarr'
probe1_compressed_folder = dst_folder + '/probe1_raw_compressed.zarr'
probe0_zarr = si.read_zarr(probe0_compressed_folder)
#probe0_zarr = si.concatenate_recordings([probe0_zarr])
job_kwargs = dict(n_jobs=40, chunk_duration='1s', progress_bar=True)
probe0_zarr.save(folder=dst_folder + '/probe0_decompressed', format='binary', **job_kwargs)

write_binary_recording 
n_jobs=36 - samples_per_chunk=29,999 - chunk_memory=21.97 MiB - total_memory=790.99 MiB - chunk_duration=1.00s (999.97 ms)


write_binary_recording: 100%|██████████████████████████████████████| 12376/12376 [46:15<00:00,  4.46it/s]


BinaryFolderRecording: 384 channels - 30.0kHz - 5 segments - 371,182,730 samples 
                       12,372.79s (3.44 hours) - int16 dtype - 265.49 GiB
Segments:
Samples:   4,665,403 | 54,043,723 | 230,016,880 | 63,209,978 | 19,246,746
Durations: 155.51s (2.59 minutes) | 1,801.46s (30.02 minutes) | 7,667.25s (2.13 hours) | 2,107.00s (35.12 minutes) | 641.56s (10.69 minutes)
Memory:    3.34 GiB | 38.66 GiB | 164.52 GiB | 45.21 GiB | 13.77 GiB

In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M24072'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[[20241204]]
base_folder='/mnt/rds01/ibn-vision/DATA/SUBJECTS/' 
no_probe = 1
# slice off the reocrding that had failing errors happened during the recording
acquisition_to_slice = []
slice_time_ranges = []
no_dates = len(save_date_list)


date_count = 0
for save_date_tmp in save_date_list:
    save_date = str(save_date_tmp[0])
    
    acquisition_folders = []
    acquisition_base_path = base_folder + mouse + '/ephys/' + save_date + '/*' + save_date
    print(acquisition_base_path)
    acquisition_folders = glob.glob(acquisition_base_path + '_*')
    print('acquisition_folders: ', acquisition_folders)

    for probe in range(int(no_probe)):
        for acquisition_folder in acquisition_folders:
                # Rename all tcat files to t0 if they exist
            tcat_pattern = os.path.join(acquisition_folder,'**','*tcat.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            # Step 1: Iterate over the list of files with tcat in the name
            for old_name in files_to_rename:
                # Step 2: Construct the new filename (REMEMBER to switch the name back to tcat)
                new_name = old_name.replace('tcat', 't0')
                
                # Step 3: Rename the file
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            
            recording_raw = si.read_spikeglx(acquisition_folder,stream_name='imec' + str(probe) + '.ap')
            print('reading recording from:', acquisition_folder,'and probe',str(probe))
            if any(f'_{acq}' in acquisition_folder for acq in acquisition_to_slice):
                print(f'Acquisition {acquisition_folder} corresponds to one of the acquisition_to_slice: {acquisition_to_slice}')
                matching_acquisition = next((acq for acq in acquisition_to_slice if f'_{acq}' in acquisition_folder), None)
                slice_index = acquisition_to_slice.index(matching_acquisition)
                time_window = slice_time_ranges[slice_index]
                print(f'Time window for acquisition {matching_acquisition}: {time_window}')
                if matching_acquisition is not None:
                    print(f'Matching acquisition: {matching_acquisition}')
                print('slicing recording to:', time_window)

                recording_raw =recording_raw.frame_slice( start_frame=time_window[0], end_frame=int(time_window[1] * 30000) )
            print(recording_raw)
            compression_folder = base_folder + mouse + '/ephys/' + save_date + '/probe' + str(probe) + '_compressed' + acquisition_folder[-2:]
            print('compressing to folder: '+compression_folder)
            raw_compressed = recording_raw.save(format="zarr", folder=compression_folder, compressor=compressor_wv, n_jobs=16, chunk_duration="1s")
            tcat_pattern = os.path.join(acquisition_folder, '**', '*t0.imec*.lf*')
            files_to_rename = glob.glob(tcat_pattern, recursive=True)
            for old_name in files_to_rename:
                new_name = old_name.replace('t0', 'tcat')    
                os.rename(old_name, new_name)
                print(f'Renamed {old_name} to {new_name}')
            load_compressed = si.read_zarr(compression_folder+'.zarr')
            recording_raw.get_probe().to_dataframe()
            fig, ax = plt.subplots(figsize=(15, 10))
            si.plot_probe_map(recording_raw, ax=ax)
            ax.set_ylim(-100, 4000)
    date_count = date_count + 1



In [ ]:
#test probe layout changing

from pathlib import Path
import os
import shutil
import numpy as np
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 
import docker
from datetime import datetime
import itertools
import scipy.io as sio
import sys
import glob
import ast
import matplotlib.pyplot as plt
from wavpack_numcodecs import WavPack
import probeinterface as pi
compressor_wv = WavPack(level=3, bps=None)
# The first command-line argument after the script name is the mouse identifier.
mouse='M24072'
# All command-line arguments after `mouse` and before `save_date` are considered dates.

save_date_list=[[20241204]]
base_folder='/mnt/rds01/ibn-vision/DATA/SUBJECTS/' 
no_probe = 1
# slice off the reocrding that had failing errors happened during the recording
acquisition_to_slice = []
slice_time_ranges = []
no_dates = len(save_date_list)


date_count = 0

#load the recording
recording_zarr = si.load_extractor(file_path='/mnt/rds01/ibn-vision/DATA/SUBJECTS/M24072/ephys/20241204/probe0_compressed_0.zarr')
#load the probe layout and plot it
recording_zarr.get_probe().to_dataframe()
fig, ax = plt.subplots(figsize=(15, 10))
si.plot_probe_map(recording_zarr, ax=ax)
ax.set_ylim(-100, 4000)

# load other metafile to get another probe layout
meta_path = '/mnt/rds01/ibn-vision/DATA/SUBJECTS/M24072/ephys/20241126/20241126_0/20241126_0_g0/20241126_0_g0_imec0/20241126_0_g0_t0.imec0.ap.meta'
new_prob = si.read_spikeglx(meta_path)
#read_imro(file_path: Union[str, Path])
#change the probe layout
recording_zarr.set_probe(new_prob.get_probe())
#plot the new probe layout
fig, ax = plt.subplots(figsize=(15, 10))
si.plot_probe_map(recording_zarr, ax=ax)
ax.set_ylim(-100, 6000)
